In [1]:
import pandas as pd
import time
from alpha_vantage.techindicators import TechIndicators
from datetime import datetime, timedelta



In [2]:
# 替换为你的API密钥
api_key = '2HPUREQ72YO9MTMW'

# 创建TechIndicators对象
ti = TechIndicators(key=api_key, output_format='pandas')

# 定义获取RSI数据的函数
def get_rsi_data(symbol, interval, time_period, series_type, start_date, end_date):
    all_data = pd.DataFrame()
    current_date = start_date
    
    while current_date < end_date:
        try:
            # 计算月份
            month_str = current_date.strftime('%Y-%m')
            
            # 获取RSI数据
            data, meta_data = ti.get_rsi(symbol=symbol, interval=interval, time_period=time_period, series_type=series_type, month=month_str)
            data['date'] = pd.to_datetime(data.index)
            data = data[(data['date'] >= current_date.strftime('%Y-%m-%d')) & 
                        (data['date'] < (current_date + timedelta(days=30)).strftime('%Y-%m-%d'))]
            all_data = pd.concat([all_data, data])
            
            # 更新日期
            current_date += timedelta(days=30)
            
            # 避免API调用频率限制
            time.sleep(12)  # Alpha Vantage限制每分钟最多5次请求
        except Exception as e:
            print(f"Error fetching data for {current_date}: {e}")
            # 等待一段时间后重试
            time.sleep(60)
    
    return all_data



In [ ]:
# 定义参数
symbol = 'NASDAQ100'  # 纳斯达克100指数代码
start_date = datetime.now() - timedelta(days=365*5)
end_date = datetime.now()

# 获取不同时间间隔的RSI数据
rsi_1min = get_rsi_data(symbol, interval='1min', time_period=14, series_type='close', start_date=start_date, end_date=end_date)
rsi_5min = get_rsi_data(symbol, interval='5min', time_period=14, series_type='close', start_date=start_date, end_date=end_date)
rsi_30min = get_rsi_data(symbol, interval='30min', time_period=14, series_type='close', start_date=start_date, end_date=end_date)
rsi_1h = get_rsi_data(symbol, interval='60min', time_period=14, series_type='close', start_date=start_date, end_date=end_date)
rsi_4h = get_rsi_data(symbol, interval='60min', time_period=14, series_type='close', start_date=start_date, end_date=end_date)

# 合并数据
all_rsi = rsi_1min.join(rsi_5min, lsuffix='_1min', rsuffix='_5min')
all_rsi = all_rsi.join(rsi_30min, rsuffix='_30min')
all_rsi = all_rsi.join(rsi_1h, rsuffix='_1h')
all_rsi = all_rsi.join(rsi_4h, rsuffix='_4h')

# 保存数据到CSV文件
all_rsi.to_csv(f'{symbol}_5year_rsi_data.csv')
print(f'Data saved to {symbol}_5year_rsi_data.csv')
print(all_rsi.head())